# Import des librairies nécessaires

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

import opendatasets as od

# Définition du modèle de convolution

In [19]:
# Vérifier si CUDA est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Définir le modèle CNN
class SkinLesionCNN(nn.Module):
    def __init__(self):
        super(SkinLesionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128*28*28, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = torch.relu(self.conv3(x))
        x = torch.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 128*28*28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


cuda


# Préparation des données

In [9]:
dataset_url = 'https://www.kaggle.com/drscarlat/melanoma'
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle username:Your Kaggle Key:Downloading melanoma.zip to .\melanoma


100%|██████████| 5.26G/5.26G [03:14<00:00, 29.0MB/s]


In [20]:
# Définir les transformations pour les données
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Télécharger le jeu de données Skin Lesion de torchvision
train_dataset = datasets.ImageFolder(root="./melanoma/DermMel/train_sep", transform=transform)
test_dataset = datasets.ImageFolder(root="./melanoma/DermMel/test", transform=transform)

# Créer les DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Entraînement du modèle

In [21]:
# Instancier le modèle
model = SkinLesionCNN().to(device)

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement du modèle
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Déplacer les données sur le GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Accuracy: {train_accuracy}%")

Epoch [0/10], Loss: 0.5233005020254387, Accuracy: 77.44804343755851%
Epoch [1/10], Loss: 0.3648453187755125, Accuracy: 84.07601572739188%
Epoch [2/10], Loss: 0.30147922811973954, Accuracy: 86.99681707545403%
Epoch [3/10], Loss: 0.2396994861634727, Accuracy: 90.1235723647257%
Epoch [4/10], Loss: 0.17890345531064059, Accuracy: 92.59501965923984%
Epoch [5/10], Loss: 0.12857331656767224, Accuracy: 95.14135929601198%
Epoch [6/10], Loss: 0.08951542530526703, Accuracy: 96.78899082568807%
Epoch [7/10], Loss: 0.06481787853338895, Accuracy: 97.55663733383261%
Epoch [8/10], Loss: 0.04367748376617799, Accuracy: 98.3149222991949%
Epoch [9/10], Loss: 0.03950864040064674, Accuracy: 98.73619172439618%


# Evaluation du modèle

In [34]:
# Évaluation du modèle
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Déplacer les données sur le GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total}%")


Accuracy on test set: 87.16652625666947%


In [35]:
model.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[-0.1445, -0.0443,  0.1019],
                        [ 0.0808,  0.1405, -0.1573],
                        [ 0.0745, -0.1121,  0.0809]],
              
                       [[-0.1226, -0.2120, -0.0813],
                        [ 0.0978,  0.0953, -0.0641],
                        [-0.0300, -0.1231, -0.1509]],
              
                       [[ 0.1316, -0.0631,  0.1317],
                        [ 0.1835, -0.0759, -0.0912],
                        [-0.0046, -0.0078,  0.1086]]],
              
              
                      [[[-0.0057,  0.0271, -0.0648],
                        [ 0.0516,  0.1125, -0.1013],
                        [-0.0500,  0.1040, -0.2252]],
              
                       [[ 0.1009,  0.0169,  0.0995],
                        [-0.0777,  0.1082, -0.1458],
                        [ 0.0666,  0.0097, -0.0778]],
              
                       [[ 0.0022, -0.0152, -0.1812],
                        [ 

# Enregistrement du modèle

In [36]:
from pathlib import Path

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = "01_SkinCancerDetectionModel.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

Saving model to: models\01_SkinCancerDetectionModel.pth
